Figure 5 is a sensitivity analysis across various model parameter variations

In [1]:
include(joinpath("..","src","setup.jl"))

┌ Warning: Package RCall does not have AxisArrays in its dependencies:
│ - If you have RCall checked out for development and have
│   added AxisArrays as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with RCall
│ Loading AxisArrays into RCall from project dependency, future warnings for RCall are suppressed.
└ @ nothing nothing:840


means = 3×2 DataFrame
│ Row │ st     │ streaming │
│     │ Int64⍰ │ Float64   │
├─────┼────────┼───────────┤
│ 1   │ 3      │ 0.134322  │
│ 2   │ 6      │ 0.588438  │
│ 3   │ 12     │ 0.92684   │


┌ Warning: combine(gd) is deprecated, use DataFrame(gd) instead
│   caller = |>(::GroupedDataFrame{DataFrame}, ::typeof(combine)) at operators.jl:813
└ @ Base ./operators.jl:813
┌ Info: Loading DataFrames support into Gadfly.jl
└ @ Gadfly /Users/davidlittle/.julia/packages/Gadfly/ew1SM/src/mapping.jl:228


RObject{VecSxp}
$rcalljl_options
$rcalljl_options$height
[1] 360

$rcalljl_options$width
[1] 432




In [2]:
fields = [:f_c_a,:f_c_m,:f_c_σ,:s_c_a,:s_c_m,:s_c_σ,:t_c_a,:t_c_m,:t_c_σ,
          :f_τ_a,:f_τ_m,:s_τ_a,:s_τ_m,:t_τ_a,:t_τ_m,
          :f_W_m_σ,:s_W_m_σ,:t_W_m_σ_t,:t_W_m_σ_ϕ,:t_W_m_σ_N];

In [3]:
param_sets = []
data_sets = []

0-element Array{Any,1}

## Time constant data set

In [4]:
datadir = joinpath("..","data","count_lengths","run_2018-12-06")

params = load_params(joinpath(datadir,"params.jld2"))
params[:pindex] = 1:size(params,1)
settings = joinpath("..","src","settings.toml")

results = []
for_results_in(joinpath(datadir,"data")) do entry
  push!(results,DataFrame(length=entry["lengths"],
                          percepts=entry["percepts"].+1, # after +1, indicates the number of streams reported, 1 or 2
                          created=entry["created"],
                          pindex=entry["pindex"])) # the parameter index (pindex = N correspondes to row N of `params`)
end
df = vcat(results...);

Progress: 100%|█████████████████████████████████████████| Time: 0:00:08


In [5]:
# turns out there are some non-unique parameter values with this; remove those (in the future do it before running the simulation)
paramssel = unique!(params,[fields;:Δf])
dfsel = @where(df,in.(:pindex,Ref(paramssel.pindex)));

paramssel[:old_pindex] = paramssel.pindex;
dfsel[:old_pindex] = dfsel.pindex;

paramssel[:pindex] = 1:size(paramssel,1)
for row in 1:size(dfsel,1)
    i = findfirst(isequal(dfsel.old_pindex[row]),paramssel.old_pindex)
    if i isa Nothing
        error("Can't find old_pindex $(dfsel.old_pindex[row])")
    end
    dfsel.pindex[row] = paramssel.pindex[i]
end
head(dfsel)

delete!(dfsel,:old_pindex);
delete!(paramssel,:old_pindex);

print(maximum(paramssel.pindex))
size(paramssel)

┌ Warning: `head(df::AbstractDataFrame)` is deprecated, use `first(df, 6)` instead.
│   caller = top-level scope at In[5]:16
└ @ Core In[5]:16
┌ Warning: `delete!(df::AbstractDataFrame, cols::Any)` is deprecated, use `deletecols!(df, cols)` instead.
│   caller = top-level scope at In[5]:17
└ @ Core In[5]:17


1575

┌ Warning: `delete!(df::AbstractDataFrame, cols::Any)` is deprecated, use `deletecols!(df, cols)` instead.
│   caller = top-level scope at In[5]:19
└ @ Core In[5]:19


(1575, 36)

In [6]:
length(unique(dfsel.pindex))

1575

In [7]:
push!(param_sets,paramssel);
push!(data_sets,dfsel);

## Noise magnitude data set

In [8]:
datadir = joinpath(datadir,"..","run_2018-12-14")
params = load_params(joinpath(datadir,"params.jld2"))
params[:pindex] = 1:size(params,1)

results = []
for_results_in(joinpath(datadir,"data")) do entry
  push!(results,DataFrame(length=entry["lengths"],
                          percepts=entry["percepts"].+1, # after +1, indicates the number of streams reported, 1 or 2
                          created=entry["created"],
                          pindex=entry["pindex"])) # the parameter index (pindex = N correspondes to row N of `params`)
end
df = vcat(results...);

Progress: 100%|█████████████████████████████████████████| Time: 0:00:03


In [9]:
push!(param_sets,params);
push!(data_sets,df);

## Inhibition breadth data set

In [10]:
datadir = joinpath(datadir,"..","run_2019-01-02")
params = load_params(joinpath(datadir,"params.jld2"))
params[:pindex] = 1:size(params,1)

results = []
for_results_in(joinpath(datadir,"data")) do entry
  push!(results,DataFrame(length=entry["lengths"],
                          percepts=entry["percepts"].+1, # after +1, indicates the number of streams reported, 1 or 2
                          created=entry["created"],
                          pindex=entry["pindex"])) # the parameter index (pindex = N correspondes to row N of `params`)
end
df = vcat(results...);

Progress: 100%|█████████████████████████████████████████| Time: 0:00:04


In [11]:
push!(param_sets,params);
push!(data_sets,df);

## Combine data sets

In [12]:
for i in 2:length(param_sets)
    @show offset = maximum(param_sets[i-1].pindex)
    param_sets[i].pindex .+= offset
    data_sets[i].pindex .+= offset
end
    
df = vcat(data_sets...);
params = vcat(param_sets...);

offset = maximum((param_sets[i - 1]).pindex) = 1575
offset = maximum((param_sets[i - 1]).pindex) = 2250


In [13]:
@assert length(unique(df.pindex)) == size(params,1)

## Summarize data

In [14]:
progress = Progress(length(groupby(params,fields)))
herr = human_error()
df_summary = by(params,fields) do row
    ## next!(progress)
    err = model_error(df,params;(k => row[1,k] for k in fields)...)
    @assert row.Δf == [3,6,12]
    df3 = df[df.pindex .== row[1,:pindex][1],:]
    df6 = df[df.pindex .== row[2,:pindex][1],:]
    df12 = df[df.pindex .== row[3,:pindex][1],:]
    
    DataFrame(stream_error = err.stream,
              length_error = err.lengths,
              stream3 = streamprop(df3.percepts,df3.length),
              stream6 = streamprop(df6.percepts,df6.length),
              stream12 = streamprop(df12.percepts,df12.length),
              eratio = error_ratio(err,herr))
end
head(df_summary)

means = 3×2 DataFrame
│ Row │ st     │ streaming │
│     │ Int64⍰ │ Float64   │
├─────┼────────┼───────────┤
│ 1   │ 3      │ 0.134322  │
│ 2   │ 6      │ 0.588438  │
│ 3   │ 12     │ 0.92684   │


┌ Warning: combine(gd) is deprecated, use DataFrame(gd) instead
│   caller = |>(::GroupedDataFrame{DataFrame}, ::typeof(combine)) at operators.jl:813
└ @ Base ./operators.jl:813
┌ Warning: `head(df::AbstractDataFrame)` is deprecated, use `first(df, 6)` instead.
│   caller = top-level scope at In[14]:18
└ @ Core In[14]:18


,f_c_a,f_c_m,f_c_σ,s_c_a,s_c_m,s_c_σ,t_c_a,t_c_m,t_c_σ,f_τ_a,f_τ_m,s_τ_a,s_τ_m,t_τ_a,t_τ_m,f_W_m_σ,s_W_m_σ,t_W_m_σ_t,t_W_m_σ_ϕ,t_W_m_σ_N,stream_error,length_error,stream3,stream6,stream12,eratio
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Quantity…,Quantity…,Quantity…,Quantity…,Quantity…,Quantity…,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,1.0 s,0.35 s,3.0 s,0.35 s,3.0 s,0.35 s,5.6,15.0,7.0,7.0,3.0,0.223436,0.215894,0.0649255,0.95678,1.0,1.12263
2,0.0,5.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,1.0 s,0.35 s,3.0 s,0.35 s,3.0 s,0.35 s,5.6,15.0,7.0,7.0,3.0,0.240384,0.479242,0.0488483,0.983754,1.0,1.58126
3,0.0,63.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,1.0 s,0.35 s,3.0 s,0.35 s,3.0 s,0.35 s,5.6,15.0,7.0,7.0,3.0,0.252312,0.999453,0.0251959,1.0,1.0,2.41043
4,0.0,790.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,1.0 s,0.35 s,3.0 s,0.35 s,3.0 s,0.35 s,5.6,15.0,7.0,7.0,3.0,0.253495,0.999453,0.0,1.0,1.0,2.41464
5,0.0,10000.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,1.0 s,0.35 s,3.0 s,0.35 s,3.0 s,0.35 s,5.6,15.0,7.0,7.0,3.0,0.253371,0.999453,0.000759713,1.0,1.0,2.4142
6,5.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,1.0 s,0.35 s,3.0 s,0.35 s,3.0 s,0.35 s,5.6,15.0,7.0,7.0,3.0,0.26239,0.330388,0.00178035,0.179899,0.729756,1.43457


In [15]:
dfplot = @linq df_summary |>
    rename_levels_for([:stream_error,:length_error,:eratio,:stream3,:stream6,:stream12],suffixes=[:c_a,:c_m,:τ_a,:τ_m,:c_σ,:W_m_σ,:W_m_σ_t,:W_m_σ_ϕ,:W_m_σ_N]) |>
    transform(a = packaxes(:c_a), m = packaxes(:c_m), rand = randn(length(:c_a))) 
              
head(dfplot)

┌ Warning: `head(df::AbstractDataFrame)` is deprecated, use `first(df, 6)` instead.
│   caller = top-level scope at In[15]:4
└ @ Core In[15]:4


,c_a,c_m,τ_a,τ_m,c_σ,W_m_σ,W_m_σ_t,W_m_σ_ϕ,W_m_σ_N,level,stream_error,length_error,eratio,stream3,stream6,stream12,a,m,rand
,Float64,Float64,Quantity…,Quantity…,Float64,Float64,Float64,Float64,Float64,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,0.0,1.0 s,0.35 s,0.2,5.6,NaN,NaN,NaN,Peripheral,0.223436,0.215894,1.12263,0.0649255,0.95678,1.0,0.0,0.0,0.45072
2,0.0,5.0,1.0 s,0.35 s,0.2,5.6,NaN,NaN,NaN,Peripheral,0.240384,0.479242,1.58126,0.0488483,0.983754,1.0,0.0,1.5,1.76249
3,0.0,63.0,1.0 s,0.35 s,0.2,5.6,NaN,NaN,NaN,Peripheral,0.252312,0.999453,2.41043,0.0251959,1.0,1.0,0.0,2.5,-0.285333
4,0.0,790.0,1.0 s,0.35 s,0.2,5.6,NaN,NaN,NaN,Peripheral,0.253495,0.999453,2.41464,0.0,1.0,1.0,0.0,3.5,0.418501
5,0.0,10000.0,1.0 s,0.35 s,0.2,5.6,NaN,NaN,NaN,Peripheral,0.253371,0.999453,2.4142,0.000759713,1.0,1.0,0.0,5.0,0.068234
6,5.0,0.0,1.0 s,0.35 s,0.2,5.6,NaN,NaN,NaN,Peripheral,0.26239,0.330388,1.43457,0.00178035,0.179899,0.729756,1.5,0.0,-0.0617323


In [16]:
dfplot[:variation] = "Aunknown"
dfplot[:varvalue] = NaN
allsel = []
sel = findall(!isequal(3.0s),dfplot.τ_a)
dfplot[sel,:variation] = "Bτ_a"
dfplot[sel,:varvalue] = ustrip.(uconvert.(s,dfplot[sel,:τ_a]))
@assert length(sel) .== 225
append!(allsel,sel)

sel = findall(!isequal(350ms),dfplot.τ_m)
dfplot[sel,:variation] = "Cτ_m"
dfplot[sel,:varvalue] = ustrip.(uconvert.(s,dfplot[sel,:τ_m]))
@assert length(sel) .== 225
append!(allsel,sel)

sel = findall(!isequal(0.2),dfplot.c_σ)
dfplot[sel,:variation] = "Dc_σ"
dfplot[sel,:varvalue] = dfplot[sel,:c_σ]
@assert length(sel) .== 225
append!(allsel,sel)

sel = findall((dfplot.W_m_σ .!= 5.6) .& (dfplot.level .== "Peripheral") .& .!isnan.(dfplot.W_m_σ))
@assert length(sel) .== 50
dfplot[sel,:variation] = "W_m_σ"
dfplot[sel,:varvalue] = dfplot[sel,:W_m_σ]/5.6

sel = findall((dfplot.W_m_σ .!= 15) .& (dfplot.level .== "Cortical") .& .!isnan.(dfplot.W_m_σ))
@assert length(sel) .== 50
dfplot[sel,:variation] = "W_m_σ"
dfplot[sel,:varvalue] = dfplot[sel,:W_m_σ]/15

sel = findall((dfplot.W_m_σ_t .!= 7.0) .& (dfplot.level .== "Object") .& .!isnan.(dfplot.W_m_σ_t))
@assert length(sel) .== 50
dfplot[sel,:variation] = "W_m_σ"
dfplot[sel,:varvalue] = dfplot[sel,:W_m_σ_t]/7.0

sel = findall((dfplot.W_m_σ_ϕ .!= 7.0) .& (dfplot.level .== "Object") .& .!isnan.(dfplot.W_m_σ_ϕ))
@assert length(sel) .== 50
dfplot[sel,:variation] = "Zdiscard"
dfplot[sel,:varvalue] = dfplot[sel,:W_m_σ_ϕ]

sel = findall((dfplot.W_m_σ_N .!= 3.0) .& (dfplot.level .== "Object") .& .!isnan.(dfplot.W_m_σ_N))
@assert length(sel) .== 50
dfplot[sel,:variation] = "W_m_σ2"
dfplot[sel,:varvalue] = dfplot[sel,:W_m_σ_N]/3.0;

In [17]:
sel = findall((dfplot.W_m_σ .!= 5.6) .& (dfplot.level .!= "Object"))
length(sel)

350

In [18]:
dfsumplot = by(dfplot,[:variation,:varvalue,:level]) do df
    lower, upper = dbootconf(shuffle(df.eratio))
    thresh = 1.35
    
    DataFrame(eratio = mean(df.eratio),
              min_eratio = minimum(df.eratio),
              min_stream_error = minimum(df.stream_error),
              min_length_error = minimum(df.length_error),
              stream3 = mean(df.stream3[df.eratio .< thresh]),
              stream6 = mean(df.stream6[df.eratio .< thresh]),
              stream12 = mean(df.stream12[df.eratio .< thresh]),
              emin = minimum(df.eratio),
              emax = maximum(df.eratio),
              elower = lower, #quantile(df.eratio,0.025),
              eupper = upper) #quantile(df.eratio,0.975))
end;

In [19]:
last(sort!(dfsumplot,:variation),25)

,variation,varvalue,level,eratio,min_eratio,min_stream_error,min_length_error,stream3,stream6,stream12,emin,emax,elower,eupper
,String,Float64,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,Cτ_m,0.15,Cortical,1.70963,0.774709,0.061914,0.159879,0.110453,0.653348,0.815355,0.774709,2.79615,1.4657,1.95017
2,Cτ_m,0.5,Cortical,1.63778,0.777696,0.0617892,0.212544,0.110442,0.64018,0.874917,0.777696,2.76216,1.38219,1.90565
3,Cτ_m,1.0,Cortical,1.68368,0.934789,0.0743507,0.22986,0.0779794,0.738959,0.871795,0.934789,2.76216,1.4359,1.95259
4,Cτ_m,0.15,Object,1.35829,0.669888,0.0600493,0.26233,0.0770106,0.566042,0.906787,0.669888,1.92752,1.19013,1.52309
5,Cτ_m,0.5,Object,1.19634,0.679492,0.0671892,0.246467,0.0860137,0.550246,0.884807,0.679492,1.90133,1.03861,1.36061
6,Cτ_m,1.0,Object,1.15421,0.632059,0.0590004,0.239666,0.0815642,0.534502,0.90088,0.632059,1.85047,1.00205,1.30047
7,Dc_σ,0.4,Peripheral,2.07266,0.679719,0.0722452,0.177945,0.0594268,0.560006,0.990543,0.679719,2.93866,1.75419,2.36482
8,Dc_σ,0.8,Peripheral,1.97617,0.668113,0.0939312,0.215409,0.0812832,0.601878,0.92663,0.668113,2.76233,1.68324,2.27848
9,Dc_σ,1.2,Peripheral,1.90524,0.730998,0.0940793,0.24231,0.101066,0.6587,0.889851,0.730998,2.76216,1.58097,2.23313


In [20]:
uniqueinds(x) = indexin(x,unique(x))
dfsumplot.xval = 3.2.*Float64.(uniqueinds(dfsumplot.variation))
for g in groupby(dfsumplot,:variation)
    sel = findall(isequal(first(g.variation)),dfsumplot.variation)
    dfsumplot[sel,:xval] = dfsumplot[sel,:xval] .+ (0.9.*uniqueinds(g.varvalue))
end
dfsumplot.xval .+= 0.3.*uniqueinds(dfsumplot.level)
dfsumplot.xval .-= 2.5

dfsumplot

,variation,varvalue,level,eratio,min_eratio,min_stream_error,min_length_error,stream3,stream6,stream12,emin,emax,elower,eupper,xval
,String,Float64,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,Aunknown,NaN,Peripheral,2.22983,0.669473,0.070641,0.192394,0.0515423,0.575345,1.0,0.669473,2.93711,1.96398,2.49258,1.9
2,Aunknown,NaN,Cortical,1.6746,0.939334,0.115874,0.249914,0.137188,0.636552,0.843243,0.939334,2.76216,1.45703,1.91356,2.2
3,Aunknown,NaN,Object,1.22814,0.6374,0.0619701,0.255804,0.0855718,0.576898,0.903244,0.6374,1.84181,1.07889,1.38469,2.5
4,Bτ_a,1.0,Peripheral,2.30685,0.963265,0.110688,0.201831,0.0352937,0.813913,1.0,0.963265,2.8394,2.04611,2.5194,5.1
5,Bτ_a,5.0,Peripheral,2.11169,0.556935,0.0808423,0.112872,0.0492833,0.434474,0.880271,0.556935,2.8081,1.8252,2.3566,6.0
6,Bτ_a,10.0,Peripheral,2.1064,0.721466,0.113091,0.127135,0.111105,0.492203,0.867065,0.721466,2.90069,1.78326,2.40057,6.9
7,Bτ_a,1.0,Cortical,1.71828,0.858246,0.102004,0.288679,0.0797456,0.625045,0.84032,0.858246,2.76301,1.52589,1.90538,5.4
8,Bτ_a,5.0,Cortical,1.64122,0.893668,0.0766466,0.22934,0.177212,0.631967,0.853343,0.893668,2.78527,1.38789,1.90209,6.3
9,Bτ_a,10.0,Cortical,1.57913,0.923276,0.0825287,0.129818,0.211011,0.630236,0.865793,0.923276,2.76216,1.43222,1.75033,7.2


In [21]:
using WebIO
node(:div, "Hello, World")

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/05ed38e1b1a66c8d039b3f080b8b49b07b6c0144-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/05ed38e1b1a66c8d039b3f080b8b49b07b6c0144-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

(div
  "Hello, World")

In [22]:
using TableView
showtable(dfsumplot)

Scope("scope-efadea4c-b344-4192-b573-50f474e64696", Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[], Dict{Symbol,Any}(:id=>"grid-ffb885a0",:className=>"ag-theme-balham",:style=>Dict("height"=>"500px","width"=>"100%")), 0), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}(), Set(String[]), nothing, Any["/Users/davidlittle/.julia/packages/TableView/sHktV/deps/ag-grid-20.1.0/ag-grid.js", "/Users/davidlittle/.julia/packages/TableView/sHktV/deps/ag-grid-20.1.0/ag-grid.css", "/Users/davidlittle/.julia/packages/TableView/sHktV/deps/ag-grid-20.1.0/ag-grid-light.css", "/Users/davidlittle/.julia/packages/TableView/sHktV/deps/ag-grid-20.1.0/ag-grid-dark.css"], Dict{Any,Any}("_promises"=>Dict{Any,Any}("importsLoaded"=>Any[JSString("(function (agGrid){var gridOptions={\"enableColResize\":true,\"columnDefs\":[{\"headerName\":\"variation\",\"headerTooltip\":\"String\",\"field\":\"variation\",\"type\":null,\"filter\":null},{\"headerName\":\"varvalue\",\"headerTooltip\":\"Float64\",\"field\":\"varvalue\",\"type\":\"numericColumn\",\"filter\":\"agNumberColumnFilter\"},{\"headerName\":\"level\",\"headerTooltip\":\"String\",\"field\":\"level\",\"type\":null,\"filter\":null},{\"headerName\":\"eratio\",\"headerTooltip\":\"Float64\",\"field\":\"eratio\",\"type\":\"numericColumn\",\"filter\":\"agNumberColumnFilter\"},{\"headerName\":\"min_eratio\",\"headerTooltip\":\"Float64\",\"field\":\"min_eratio\",\"type\":\"numericColumn\",\"filter\":\"agNumberColumnFilter\"},{\"headerName\":\"min_stream_error\",\"headerTooltip\":\"Float64\",\"field\":\"min_stream_error\",\"type\":\"numericColumn\",\"filter\":\"agNumberColumnFilter\"},{\"headerName\":\"min_length_error\",\"headerTooltip\":\"Float64\",\"field\":\"min_length_error\",\"type\":\"numericColumn\",\"filter\":\"agNumberColumnFilter\"},{\"headerName\":\"stream3\",\"headerTooltip\":\"Float64\",\"field\":\"stream3\",\"type\":\"numericColumn\",\"filter\":\"agNumberColumnFilter\"},{\"headerName\":\"stream6\",\"headerTooltip\":\"Float64\",\"field\":\"stream6\",\"type\":\"numericColumn\",\"filter\":\"agNumberColumnFilter\"},{\"headerName\":\"stream12\",\"headerTooltip\":\"Float64\",\"field\":\"stream12\",\"type\":\"numericColumn\",\"filter\":\"agNumberColumnFilter\"},{\"headerName\":\"emin\",\"headerTooltip\":\"Float64\",\"field\":\"emin\",\"type\":\"numericColumn\",\"filter\":\"agNumberColumnFilter\"},{\"headerName\":\"emax\",\"headerTooltip\":\"Float64\",\"field\":\"emax\",\"type\":\"numericColumn\",\"filter\":\"agNumberColumnFilter\"},{\"headerName\":\"elower\",\"headerTooltip\":\"Float64\",\"field\":\"elower\",\"type\":\"numericColumn\",\"filter\":\"agNumberColumnFilter\"},{\"headerName\":\"eupper\",\"headerTooltip\":\"Float64\",\"field\":\"eupper\",\"type\":\"numericColumn\",\"filter\":\"agNumberColumnFilter\"},{\"headerName\":\"xval\",\"headerTooltip\":\"Float64\",\"field\":\"xval\",\"type\":\"numericColumn\",\"filter\":\"agNumberColumnFilter\"}],\"multiSortKey\":\"ctrl\",\"rowData\":[{\"variation\":\"Aunknown\",\"varvalue\":null,\"level\":\"Peripheral\",\"eratio\":2.2298255226484924,\"min_eratio\":0.6694731003186911,\"min_stream_error\":0.07064102086118448,\"min_length_error\":0.19239384351528938,\"stream3\":0.05154227992257356,\"stream6\":0.575345134824644,\"stream12\":1.0,\"emin\":0.6694731003186911,\"emax\":2.93710501636942,\"elower\":1.9639790264000765,\"eupper\":2.492579750707428,\"xval\":1.9000000000000004},{\"variation\":\"Aunknown\",\"varvalue\":null,\"level\":\"Cortical\",\"eratio\":1.67459615635861,\"min_eratio\":0.9393337679868465,\"min_stream_error\":0.1158735714051218,\"min_length_error\":0.24991378832784017,\"stream3\":0.13718807258314866,\"stream6\":0.636551702442487,\"stream12\":0.843242888946668,\"emin\":0.9393337679868465,\"emax\":2.762163584128584,\"elower\":1.457034719162055,\"eupper\":1.9135562463137552,\"xval\":2.2},{\"variation\":\"Aunknown\",\"varvalue\":null,\"level\":\"Object\",\"eratio\":1.2281415246115868,\"min_eratio\":0.6374004087217895,\"min_stream_err

In [151]:
dfsumplot[occursin.(Ref(r"^O.*ϕ.*"),dfsumplot.variation),:xval]

0-element Array{Float64,1}

In [152]:
p = plot(dfsumplot,x=:xval,y=:eratio,group=:varvalue,color=:level,Geom.bar)
draw(SVG(12inch,3inch),p)
draw(PDF(joinpath(plotdir,"fig5_raw.pdf"),12inch,3inch),p)

┌ Warning: The following aesthetics are mapped, but not used by any geometry:
│ group
└ @ Gadfly /Users/davidlittle/.julia/packages/Gadfly/ew1SM/src/Gadfly.jl:484


MethodError: MethodError: no method matching unsafe_wrap(::Type{Array}, ::Ptr{UInt8})
Closest candidates are:
  unsafe_wrap(::Union{Type{Array}, Type{Array{T,N} where N}, Type{Array{T,N}}}, ::Ptr{T}, !Matched::Tuple{Vararg{Int64,N}}; own) where {T, N} at pointer.jl:84
  unsafe_wrap(::Union{Type{Array}, Type{Array{T,N} where N}, Type{Array{T,1}}}, ::Ptr{T}, !Matched::Integer; own) where T at pointer.jl:89
  unsafe_wrap(::Type, ::Ptr, !Matched::Tuple{Vararg{#s70,N}} where #s70<:Integer; own) where N at pointer.jl:92

In [38]:
p = plot(dfsumplot,x=:xval,y=:min_stream_error,group=:varvalue,color=:level,Geom.bar)
draw(SVG(12inch,3inch),p)

┌ Warning: The following aesthetics are mapped, but not used by any geometry:
│ group
└ @ Gadfly /Users/davidlittle/.julia/packages/Gadfly/ew1SM/src/Gadfly.jl:484
┌ Warning: For svg transparent colors, use either e.g. fill(RGBA(r,g,b,a)) or fillopacity(a), but not both.
└ @ Compose /Users/davidlittle/.julia/packages/Compose/wlPCt/src/svg.jl:1271


In [73]:
p = plot(dfsumplot,x=:xval,y=:min_stream_error,group=:varvalue,color=:level,Geom.bar)
draw(SVG(12inch,3inch),p)

┌ Warning: The following aesthetics are mapped, but not used by any geometry:
│ group
└ @ Gadfly /Users/davidlittle/.julia/packages/Gadfly/ew1SM/src/Gadfly.jl:484
┌ Warning: For svg transparent colors, use either e.g. fill(RGBA(r,g,b,a)) or fillopacity(a), but not both.
└ @ Compose /Users/davidlittle/.julia/packages/Compose/wlPCt/src/svg.jl:1271


In [39]:
mean(dfsumplot.min_stream_error), std(dfsumplot.min_stream_error)

(0.07894799426471197, 0.030965383101050174)

In [40]:
p = plot(dfsumplot,x=:xval,y=:min_length_error,color=:level,Geom.bar)
draw(SVG(12inch,3inch),p)

┌ Warning: For svg transparent colors, use either e.g. fill(RGBA(r,g,b,a)) or fillopacity(a), but not both.
└ @ Compose /Users/davidlittle/.julia/packages/Compose/wlPCt/src/svg.jl:1271


In [76]:
mean(dfsumplot.min_length_error), std(dfsumplot.min_length_error)

(0.1973737602710632, 0.03124902478234511)

In [82]:
mean(dfsumplot.min_eratio), std(dfsumplot.min_eratio), maximum(dfsumplot.min_eratio)

(0.8748461794648783, 0.18955331980158496, 1.316721293120378)

In [41]:
p = plot(dfsumplot,x=:xval,y=:stream6,ymin=:stream3,ymax=:stream12,group=:varvalue,color=:level,Geom.point,Geom.errorbar)
draw(SVG(12inch,1.8inch),p)

┌ Warning: The following aesthetics are mapped, but not used by any geometry:
│ group
└ @ Gadfly /Users/davidlittle/.julia/packages/Gadfly/ew1SM/src/Gadfly.jl:484
┌ Warning: For svg transparent colors, use either e.g. fill(RGBA(r,g,b,a)) or fillopacity(a), but not both.
└ @ Compose /Users/davidlittle/.julia/packages/Compose/wlPCt/src/svg.jl:1271


In [42]:
draw(SVG(6inch,8inch),plot(dfplot[dfplot.τ_a .!= 3.0s,:],x=:level,y=:eratio,ygroup=:τ_a,
        Geom.subplot_grid(Geom.beeswarm)))

┌ Warning: For svg transparent colors, use either e.g. fill(RGBA(r,g,b,a)) or fillopacity(a), but not both.
└ @ Compose /Users/davidlittle/.julia/packages/Compose/wlPCt/src/svg.jl:1271
┌ Warning: For svg transparent colors, use either e.g. fill(RGBA(r,g,b,a)) or fillopacity(a), but not both.
└ @ Compose /Users/davidlittle/.julia/packages/Compose/wlPCt/src/svg.jl:1271


┌ Warning: For svg transparent colors, use either e.g. fill(RGBA(r,g,b,a)) or fillopacity(a), but not both.
└ @ Compose /Users/davidlittle/.julia/packages/Compose/wlPCt/src/svg.jl:1271


In [43]:
draw(SVG(6inch,8inch),plot(dfplot[dfplot.τ_m .!= 350ms,:],x=:level,y=:eratio,ygroup=:τ_m,Geom.subplot_grid(Geom.beeswarm)))

┌ Warning: For svg transparent colors, use either e.g. fill(RGBA(r,g,b,a)) or fillopacity(a), but not both.
└ @ Compose /Users/davidlittle/.julia/packages/Compose/wlPCt/src/svg.jl:1271
┌ Warning: For svg transparent colors, use either e.g. fill(RGBA(r,g,b,a)) or fillopacity(a), but not both.
└ @ Compose /Users/davidlittle/.julia/packages/Compose/wlPCt/src/svg.jl:1271
┌ Warning: For svg transparent colors, use either e.g. fill(RGBA(r,g,b,a)) or fillopacity(a), but not both.
└ @ Compose /Users/davidlittle/.julia/packages/Compose/wlPCt/src/svg.jl:1271


In [44]:
draw(SVG(6inch,8inch),plot(dfplot[dfplot.c_σ .!= 0.2,:],x=:level,y=:eratio,ygroup=:c_σ,Geom.subplot_grid(Geom.beeswarm)))

┌ Warning: For svg transparent colors, use either e.g. fill(RGBA(r,g,b,a)) or fillopacity(a), but not both.
└ @ Compose /Users/davidlittle/.julia/packages/Compose/wlPCt/src/svg.jl:1271
┌ Warning: For svg transparent colors, use either e.g. fill(RGBA(r,g,b,a)) or fillopacity(a), but not both.
└ @ Compose /Users/davidlittle/.julia/packages/Compose/wlPCt/src/svg.jl:1271
┌ Warning: For svg transparent colors, use either e.g. fill(RGBA(r,g,b,a)) or fillopacity(a), but not both.
└ @ Compose /Users/davidlittle/.julia/packages/Compose/wlPCt/src/svg.jl:1271


In [ ]:
# TODO: visually verify the overall distribution for the inhibition parameters